Add stuff like installing datasets and stuf before running our app <br>
Also add different ways to run the process (Like run with completely new RNN weights, use pre-existing weights etc.)

In [1]:
import torch
from torch import nn
from ChildNetworkMLP import set_data_params, ChildModel, build_model_from_encoding, train_model, train_model_v2

from Controller import RNNController, generate_sequence, VOCAB_SIZE
from torch.utils.data import DataLoader


['START', '10', 'END']


In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [3]:
from utils import check_cifar_dataset_exists

data_path = check_cifar_dataset_exists()

In [4]:
data = torch.load(data_path + 'cifar/train_data.pt')
label = torch.load(data_path + 'cifar/train_label.pt')

C:\Users\vaibh\AppData\Local\Temp\ipykernel_13964\442911236.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  data = torch.load(data_path + 'cifar/train_data.pt')
C:\Users

In [5]:
input_dim = data[0].view(1, -1).size()[1]
output_dim = torch.max(label).item() + 1

print(input_dim)
print(output_dim)

3072
10


In [6]:
controllerModel = RNNController(VOCAB_SIZE)

In [7]:
model_encoding = generate_sequence(controllerModel)

model_encoding

['START', '7', 'START', 'END']

In [8]:
# TODO: Sanity checks before passing to generate child network

In [9]:
set_data_params(input_dim, output_dim)

In [10]:
childModel = ChildModel(model_encoding)

3072


In [11]:
childModel.state_dict()

OrderedDict([('model.0.weight',
              tensor([[ 0.0128,  0.0024,  0.0080,  ...,  0.0014,  0.0110,  0.0068],
                      [-0.0020,  0.0179,  0.0164,  ..., -0.0076,  0.0027,  0.0058],
                      [-0.0088, -0.0073, -0.0165,  ...,  0.0070, -0.0082, -0.0045],
                      ...,
                      [ 0.0103, -0.0086,  0.0155,  ..., -0.0089, -0.0103, -0.0159],
                      [ 0.0179, -0.0004, -0.0053,  ...,  0.0042, -0.0137, -0.0080],
                      [-0.0044,  0.0086, -0.0078,  ...,  0.0164, -0.0056, -0.0159]])),
             ('model.0.bias',
              tensor([ 0.0089, -0.0175, -0.0047,  0.0147, -0.0012, -0.0104, -0.0108])),
             ('model.2.weight',
              tensor([[ 0.1528, -0.2586,  0.3560,  0.2768, -0.3003,  0.1004, -0.1678],
                      [ 0.2339, -0.1506,  0.1893,  0.2155,  0.1679,  0.2057, -0.2186],
                      [-0.3616,  0.1429,  0.0563,  0.2454, -0.2683, -0.3736, -0.1503],
                      [

In [12]:
criterion = nn.CrossEntropyLoss()  # Cross-entropy loss function
optimizer = torch.optim.Adam(childModel.parameters(), lr=0.001)  # 0.001 is the learning rate

In [13]:
train_loader = DataLoader(data, batch_size=128, shuffle=True)

In [14]:
train_loss, train_time = train_model_v2(childModel, data, label, criterion, optimizer, device)

print(train_loss, train_time)

0.011106487312316895 0.8642220497131348
